# NIRCam Claw Detection.
---
**Author**: Mees Fix (mfix@stsci.edu) <br>
**Latest Update**: 05 March 2025<br>
**Use case**: NIRCam Imaging detectors A1, A3, B1, B2 or B4. <br>
**Data**: None<br>
**Test Pipeline Version**: None

<a id='intro'></a>
## Introduction

This notebook demonstrates how to use JWST pointing information provided in a user's APT program to detect the potential of
[claws](https://jwst-docs.stsci.edu/known-issues-with-jwst-data/nircam-known-issues/nircam-scattered-light-artifacts#NIRCamScatteredLightArtifacts-clawsClaws) in NIRCam observations. Claws are a scattered light phenomenon that occur when a bright star falls in the susceptibility region. When present, claws occur primarily in the A1, A3, B1, B2 or B4 detectors, affecting roughly 5% of the pixels on those detectors.

All datasets necessary for this notebook to run are included in the [jwst_rogue_path_tool](https://github.com/spacetelescope/jwst_rogue_path_tool) python package. This includes the following:

* `APT_test_4RPtool.records.json`:  A mock program that contains NIRCam instrument configurations that could contain contain claws we want to detect.
* `rogue_path_nrca.fits`: fits file that contains the susceptibility region intensity contours for NIRCam module A. 
* `rogue_path_nrcb.fits`: fits file that contains the susceptibility region intensity contours for NIRCam module B.
* `two_mass_kmag_lt_5.csv`: [2MASS](https://irsa.ipac.caltech.edu/Missions/2mass.html) stellar sources that have magnitudes less than 5.

To generate your own `records.json` file for a specific program via APT, see the [JDox page for APT Export files](https://jwst-docs.stsci.edu/jwst-astronomers-proposal-tool-overview/additional-jwst-apt-functionality/apt-export-files#APTExportFiles-records_filerecordsfile&gsc.tab=0).

<a id='top'></a>
## Table of Contents {-}
* [Imports](#imports)
* [Run the JWST Rogue Path Tool](#Run-jwst-rpt)
    * [Handling records.json Files](#handle-records-json-files)
    * [Selecting and Displaying APT Data](#Selecting-displaying-program-data)
    * [Generating APT Program Object](#Generating-apt-program-object)
* [Plotting Rogue Path Tool Results](#Plotting-rogue-path-tool-results)
    * [Exposure Level Valid Angle Plots](#exposure-level-plots)
    * [Observation Level Valid Angle Plots](#observation-level-plots)
    * [Single Angle of Attitude in V2 & V3 Space Plots](#v2-vs-v3)
    * [Flux as a Function of Attitude Plots](#attitude-vs-flux)
* [Reporting & Applying PA Constraints in APT](#reporting-and-apt)

<a id='imports'></a>
## Imports

In [ ]:
import numpy as np
import pathlib

from jwst_rogue_path_tool.program_data_parser import aptJsonFile
from jwst_rogue_path_tool.detect_claws import aptProgram
from jwst_rogue_path_tool.constants import PROJECT_DIRNAME

<a id='Run-jwst-rpt'></a>
## Run the JWST Rogue Path Tool

<a id='handle-records-json-files'></a>
### Handling records.json Files

Here we will look at the tools in the JWST Rogue Path Tool to parse and analyze the input data for the tool.

In [ ]:
apt_json_filename = pathlib.Path(PROJECT_DIRNAME) / "data" / "APT_test_4RPtool.records.json"

# Parse the APT JSON file to obtain useful information for Claw Tool
parsed_json = aptJsonFile(apt_json_filename)

parsed_json.tablenames

<a id='Selecting-displaying-program-data'></a>
### Selecting and Displaying APT Data

In [ ]:
tablename = "exposures"
parsed_json.data[tablename]  # This is a list of dictionaries

In [ ]:
exposures_df = parsed_json.build_dataframe(tablename)
exposures_df  # this is a dataframe

<a id='Generating-apt-program-object'></a>
### Generating APT Program Object

The examples below showcase the difference use cases for the JWST Rogue Path Tool.

In [ ]:
program = aptProgram(apt_json_filename, angular_step=1.0)

In [ ]:
program.run()

In [ ]:
program.observations.data  # Show ALL observation data

In [ ]:
program.observations.supported_observations  # Only show the supported observations for JWST Rogue Path Tool
program.observations.supported_observations.keys()  # Observation numbers to select data on.

In [ ]:
# Select a single observation of interest
observation = program.observations.data[1]
observation['exposure_frames'].data
# show pandas df for single exposure frame

<a id='Plotting-rogue-path-tool-results'></a>
## Plotting Rogue Path Tool Results

<a id='exposure-level-plots'></a>
### Exposure Level Valid Angle Plots

In [ ]:
supported_observations = program.observations.supported_observations.keys()
for obs_number in supported_observations:
    program.plot_exposures(program.observations.data[obs_number])

<a id='observation-level-plots'></a>
### Observation Level Valid Angle Plots
This will show valid angle ranges across all exposures in a single observation. If angles ranges are valid in one exposure but not in others, it will not be shown at this level.

In [ ]:
supported_observations = program.observations.supported_observations.keys()
for obs_number in supported_observations:
    program.plot_observation(program.observations.data[obs_number])

<a id='v2-vs-v3'></a>
### Plotting a Single Angle of Attitude in V2 & V3 Space

The figure below shows what the tool is searching for when sweeping angles. Here for a single angle we show the V2 & V3 space of the targets in our source catalog as well as the susceptibility region. The intensity of targets that fall in this region are dependent on the location where of the target in the susceptibility region and the brightness of the target.

In [ ]:
from jwst_rogue_path_tool.plotting import plot_fixed_angle_regions

angles = np.arange(100, 106)

for angle in angles:
    plot_fixed_angle_regions(observation, angle)

<a id='attitude-vs-flux'></a>
### Flux as a Function of Attitude

Below we show the calculated flux as a function of attitude angle. This figure also ties in the [JWST Backgrounds Tool](https://github.com/spacetelescope/jwst_backgrounds) to calculate the background of the target JWST is observing described in the APT file. By default, we set the parameters for the backgrounds tool to calculate 10% of the minimum background value. You can set the function and percentage via a dictionary and pass it to the AptProgram class.
~~~
jbt_params = [{"threshold":0.1, function:np.min}, {"threshold":0.2, "function":np.mean}, ..... {"threshold":value, "function":stats_function}]

AptProgram(apt_JSON_filename, angular_step=1.0, bkg_params=jbt_params)
~~~

In [ ]:
from jwst_rogue_path_tool.plotting import create_v3pa_vs_flux_plot

for obs_number in supported_observations:
    create_v3pa_vs_flux_plot(program.observations.data[obs_number])

<a id='reporting-and-apt'></a>
## Reporting & Applying PA Constraints in APT

In this section, we demonstrate how to plot the predicted claw flux across all position angles for a single observation, identify the angles free from claws, and modify the observation in APT to ensure it is taken only within those good position angle ranges.

In [ ]:
# The observation number of interest
obs_number = 5

# Make plot of claws flux vs position angle
create_v3pa_vs_flux_plot(program.observations.data[obs_number])

# Make file containing good position angles
program.make_background_report(program.observations.data[obs_number], output_directory=pathlib.Path.cwd())

The plot above illustrates the predicted claw flux at each position angle for every filter. The horizontal orange and green lines indicate signal levels at 10% (based on minimum background throughout the year) and 20% (based on mean background throughout the year) of the predicted background levels, respectively. These thresholds help identify position angles free from claw contamination. Typically, the 20% threshold is sufficient, but for certain targets—such as faint, high-redshift objects—observers may prefer the more conservative 10% threshold. The PA ranges that are free from claws based on each threshold are saved to a text file, e.g. for this example observation `program_0_observation_5_background_module_B.txt`:
```
**** Ranges Not Impacted by Background Thresholds ****
**** Module B ****
**** Ranges Under 0.1 of min  ****
PA Start -- PA End: 0 -- 80
PA Start -- PA End: 103 -- 115
PA Start -- PA End: 126 -- 157
PA Start -- PA End: 162 -- 163
PA Start -- PA End: 184 -- 221
PA Start -- PA End: 243 -- 253
PA Start -- PA End: 263 -- 265
PA Start -- PA End: 270 -- 300
PA Start -- PA End: 310 -- 312
PA Start -- PA End: 315 -- 359
**** Ranges Under 0.2 of mean  ****
PA Start -- PA End: 0 -- 81
PA Start -- PA End: 92 -- 117
PA Start -- PA End: 124 -- 165
PA Start -- PA End: 178 -- 178
PA Start -- PA End: 183 -- 223
PA Start -- PA End: 235 -- 237
PA Start -- PA End: 240 -- 302
PA Start -- PA End: 308 -- 359
```

The PA ranges in this file are determined conservatively by considering all filters together. If the predicted claw flux in any filter exceeds the threshold, the corresponding position angle is excluded from these ranges.

<br>

Once the PA ranges free from claws are determined, users can add them to their APT files as a [PA Special Requirement](https://jwst-docs.stsci.edu/jppom/special-requirements/aperture-position-angle-special-requirements#gsc.tab=0) to ensure their observations are only taken within those PA ranges. To do this, select the observation in APT, then go to `Special Requirement→Add→Position Angle→PA Range` and input each good PA range in the V3PA Range boxes. Each PA range should be entered as a separate Special Requirement. For example, using the 20% threshold, the first Special Requirement for the example observation above would be:
```
Aperture PA Range 359.92542306 to 80.92542306 Degrees (V3 0.0 to 81.0)
```

Currently, the tool does not take into account target visibility, so some of the PA ranges shown may not need to be considered. For example, if a strong claw is predicted at a PA that is never available in the [roll analysis](https://jwst-docs.stsci.edu/jwst-astronomers-proposal-tool-overview/apt-workflow-articles/apt-visit-planner#gsc.tab=0), users don't have to include Special Requirements to avoid that PA as it will never be used anyway. To check which PA ranges are actually available for a given visit, users can go to `Visit Planner→Reports→Visitx.x→Total Roll Analysis For Visit`.